In [176]:
import pandas as pd 
import numpy as np
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [177]:
from sklearn.feature_extraction.text import CountVectorizer
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer


ps = PorterStemmer()

In [178]:
file = pd.read_csv('traindata.csv', header=None, skiprows=1, usecols=[0,1])

#print(file)

In [179]:
unique_class=file[0].unique()
print(unique_class)

['science' 'sports' 'business' 'covid']


In [180]:
file.shape
total_count_train=file.shape[0]

In [181]:
def text_scaling(text):
    step = str.maketrans('', '','0,1,2,3,4,5,6,7,8,9')
    table = str.maketrans('', '', string.punctuation)
    text=text.translate(step)
    text=text.translate(table)
    word=text.split()

        #converting to lower case letters
    words = [w.lower() for w in word]
    filtered_words=[]
    #filtering the words with removing stop words
    stop_words = stopwords.words('english')
    #print(stop_words) 
    for w in words:
        if w not in stop_words:
            filtered_words.append(w)
    words=filtered_words

        #portedStemmer
    ported_sentence=[]
    for w in words:
        trial=ps.stem(w)
        ported_sentence.append(trial)
    words=ported_sentence
    
    return words


def text_word_scaling(file):
    
    class_words={}
    class_label_count={}
    total_count_class={}
    for i in range(80):
        clas=file.iloc[i,0]
        text=file.iloc[i,1]
        words=text_scaling(text)

        if clas in class_words:
            class_words[clas].extend(words)
            class_label_count[clas]+=1
            total_count_class[clas]+=len(words)
        else:
            class_words[clas] = words
            class_label_count[clas]=1
            total_count_class[clas]=len(words)




    print(class_label_count)
    print(total_count_class)
    return class_words, class_label_count, total_count_class

In [182]:
#print(class_words)
class_words, class_label_count, total_count_class=text_word_scaling(file)


{'science': 20, 'sports': 20, 'business': 19, 'covid': 21}
{'science': 200, 'sports': 420, 'business': 348, 'covid': 252}


In [183]:
list_count=[]
prior_probability={}
for k in total_count_class:
    list_count.append(total_count_class[k])
print(list_count)
for k in class_label_count:
    prior_probability[k]=class_label_count[k]/total_count_train
print(prior_probability)

[200, 420, 348, 252]
{'science': 0.25, 'sports': 0.25, 'business': 0.2375, 'covid': 0.2625}


In [184]:
def frequency(words):
    word_count = {}
    for i in words:
        if i in word_count:
            word_count[i] += 1
        else:
            word_count[i] = 1
    return word_count

def conditional_probability(frequency, count):
    class_conditional={}
    for k in frequency:
        class_conditional[k] = float(frequency[k]/count)
    return class_conditional

class_conditional_probability={}
i=0;
for k in class_words:
    class_frequency_words=frequency(class_words[k])
    conditional_prob = conditional_probability(class_frequency_words, list_count[i])
    i+=1
    class_conditional_probability[k] = conditional_prob
    #print(class_frequency_words)

In [185]:
#print(class_conditional_probability)

In [186]:
test_file = pd.read_csv('testdata.csv', header=None, skiprows=1, usecols=[0,1])
x,y = test_file.shape
print(x, y)
#print(test_file)

20 2


In [187]:
def posterior_find(test_scaled_words, class_conditional_probability, prior_probability):
    posterior_distribution={}
    probability_feature=0
    for train_class in prior_probability:
        posterior_probility = 1
        for w_test in test_scaled_words:
            class_conditional_word_probabilities= class_conditional_probability[train_class]
            if w_test not in class_conditional_word_probabilities:
                posterior_probility *= 0.001
            else:
                posterior_probility *= class_conditional_word_probabilities[w_test]
        posterior_probility *= prior_probability[train_class]
        probability_feature += posterior_probility
        posterior_distribution[train_class] = posterior_probility
    return posterior_probility, posterior_distribution, probability_feature

confidence_list=[]
predicted_list=[]
    
for i in range(x):
    text=test_file.iloc[i,1]
    test_scaled_words = text_scaling(text)
    #print(test_scaled_words)
    posterior_probility, posterior_distribution, probability_feature=posterior_find(test_scaled_words,class_conditional_probability, prior_probability)
    maximum_probability=-1
    for key in posterior_distribution:
        posterior_distribution[key] = posterior_distribution[key]/probability_feature
        if posterior_distribution[key] > maximum_probability:
            max_p_ind = key 
            maximum_probability = posterior_distribution[key]
    confidence_list.append(maximum_probability)
    predicted_list.append(max_p_ind)

In [188]:
print(predicted_list)

['science', 'science', 'science', 'science', 'science', 'sports', 'sports', 'sports', 'sports', 'sports', 'business', 'business', 'business', 'business', 'business', 'covid', 'covid', 'covid', 'covid', 'covid']


In [189]:
print(confidence_list)

[0.9999993834670468, 0.9037337995591453, 0.9998325281034195, 0.9999999526960167, 0.879421536056283, 0.999999999891396, 0.9999999999999741, 0.9999999999588761, 0.9976989909947366, 0.9998479790821991, 1.0, 0.9865961475268178, 0.8372377991538786, 0.9952624753945555, 0.9999999999997176, 0.9999819951734475, 0.9999970698650167, 0.9999996650675628, 0.9999999871736334, 0.9999907615262289]


# continuation



part-A

In [190]:
file_2 = pd.read_csv('40.csv', header=None, skiprows=1)

In [191]:
#print(file_2)

In [192]:
x,y=file_2.shape
print(x,y)
#print(file_2.iloc[2])

40 1


In [193]:
total_vocab_list=[]
split_words=[]
for i in range(x):
    text_2=file_2.iloc[i,0]
    words=text_scaling(text_2)
    split_words.append(words)
    total_vocab_list.extend(words)
print(total_vocab_list[0:20])
print(split_words[0:3])

['midst', 'covid', 'pandem', 'eat', 'healthi', 'food', 'remain', 'import', 'part', 'maintain', 'health', 'specif', 'food', 'help', 'protect', 'viru', 'nutriti', 'diet', 'boost', 'immun']
[['midst', 'covid', 'pandem', 'eat', 'healthi', 'food', 'remain', 'import', 'part', 'maintain', 'health'], ['specif', 'food', 'help', 'protect', 'viru', 'nutriti', 'diet', 'boost', 'immun', 'system', 'help', 'fight', 'symptom'], ['may', 'abl', 'share', 'meal', 'friend', 'love', 'one', 'lot', 'way', 'eat', 'well', 'support', 'health', 'difficult', 'time']]


part-B

In [194]:
freq_vocab=frequency(total_vocab_list)
n=len(total_vocab_list)
prior_2={}
for k in freq_vocab:
    prior_2[k]=freq_vocab[k]/n
print(prior_2)

{'midst': 0.0017331022530329288, 'covid': 0.0017331022530329288, 'pandem': 0.0017331022530329288, 'eat': 0.029462738301559793, 'healthi': 0.024263431542461005, 'food': 0.03292894280762565, 'remain': 0.0017331022530329288, 'import': 0.005199306759098787, 'part': 0.0017331022530329288, 'maintain': 0.0017331022530329288, 'health': 0.008665511265164644, 'specif': 0.0017331022530329288, 'help': 0.01733102253032929, 'protect': 0.0034662045060658577, 'viru': 0.0017331022530329288, 'nutriti': 0.0017331022530329288, 'diet': 0.02079722703639515, 'boost': 0.005199306759098787, 'immun': 0.0017331022530329288, 'system': 0.0034662045060658577, 'fight': 0.0017331022530329288, 'symptom': 0.0034662045060658577, 'may': 0.0017331022530329288, 'abl': 0.0017331022530329288, 'share': 0.0017331022530329288, 'meal': 0.008665511265164644, 'friend': 0.0017331022530329288, 'love': 0.005199306759098787, 'one': 0.0034662045060658577, 'lot': 0.0017331022530329288, 'way': 0.005199306759098787, 'well': 0.005199306759